In [48]:
#import matplotlib as mpl
#mpl.use('Agg') #I do not know what 'Agg' refers to.
# the .use(_) trait sets the matplotlib 'backend' (non user-facing code)
# to specify thing like what data format the images are to be ported to,
# rendered with, etc.
# The 'Agg' argument sets the image renderer to produce .png files.

#matplotlib.pyplot is imported later.

import yt
import sys
import argparse
import numpy as np
import os
from scipy.interpolate import interp1d as ip1d
import matplotlib.pyplot as plt

MSun = yt.units.Msun.in_cgs().v
cmpc = yt.units.pc.in_cgs().v
G    = yt.physical_constants.G.v
kB   = yt.physical_constants.kb.v
mH   = yt.physical_constants.mass_hydrogen_cgs.v

In [18]:
    NCD = (128,128,128)
    kmin = 1
    kmax = 32
    Eslp = 3./5.
    ssbox = np.zeros(NCD, dtype=complex)

    # first, create uniform complex random spectrum in whole array
    # range of coeffs is (-sqrt(2)/2, sqrt(2)/2), it results in
    # range (-1,1) for coeffiecient magnitudes -JW
    # This effecrively is creating the random structure on which the
    # turbulence velocities will be placed. -SL
    ssbox.real = np.sqrt(2.)*(np.random.rand(NCD[0],NCD[1],NCD[2]) - 0.5)
    ssbox.imag = np.sqrt(2.)*(np.random.rand(NCD[0],NCD[1],NCD[2]) - 0.5)

    # Givent the ssbox array, create a mask that filters selected models and
    # gives weights according to the given spectrum (Eslp).
    # This creates a mesh of values where if the spacing is less than the
    # minimum wave number and greater than the maximum, set the mesh value
    # and energy spectrum coeff (and velocity?) to zero.
    
    ax = NCD[0]/2-abs(np.arange(NCD[0], dtype=np.float64)-NCD[0]/2)
    ay = NCD[1]/2-abs(np.arange(NCD[1], dtype=np.float64)-NCD[1]/2)
    az = NCD[2]/2-abs(np.arange(NCD[2], dtype=np.float64)-NCD[2]/2)

    (mx, my, mz) = np.meshgrid(ax,ay,az)

    mask = mx*mx + my*my + mz*mz
    mask[np.where(mask < kmin*kmin)] = 0
    mask[np.where(mask > kmax*kmax)] = 0
    # E(k) ~ k^Eslp; number of modes grows with k^2 => Eslp-2
    # v ~ sqrt(E) => 0.5*Eslp-1, k^2 is in mask => 0.25*Eslp-0.5
    mask[np.where(mask>0)] = mask[np.where(mask>0)]**(0.25*Eslp-0.5)

    ssbox *= mask

In [19]:
print ssbox.min()

(-0.561263534984+0.0170778124635j)


In [34]:
def calc_sphsym_pot(r_arr, rho_arr):
    C4pio3 = 4*np.pi/3.0
    dr_arr       = np.correlate(r_arr, [-1,1])     # thicknesses of shells between r-points
    rmid_arr     = np.convolve(r_arr, [0.5,0.5])   # mid-points, rmid[0] = 0
    rmid_arr[-1] = r_arr[-1]                    # last rmid segment end at the surface
    drmid_arr    = np.correlate(rmid_arr, [-1,1])  # thicknesses of shells between mid-points
    r3mid_arr    = rmid_arr**3                  # r^3 at mid-points
    dr3mid_arr   = np.correlate(r3mid_arr, [-1,1]) # volumes of shells between mid-points
    dMrmid_arr   = rho_arr*dr3mid_arr*C4pio3    # masses of shells -"-
    Mrmid_arr    = dMrmid_arr.cumsum()          # enclosed mass below mid-points
    Mrmid_arr    = np.insert(Mrmid_arr, 0, 0.0)    # central point
    Frmid_arr    = -G*Mrmid_arr/(rmid_arr**2+1e-99) # grav. force at mid-points
    dPhi_arr     = Frmid_arr[1:-1] * dr_arr     # prepare for integration of Fr*dr
    pot_arr      = np.zeros(r_arr.shape[0], dtype=np.float64)
    pot_arr[-1]  = -G*Mrmid_arr[-1]/r_arr[-1]   # -G*M/R at the sphere surface
    pot_arr[:-1] = dPhi_arr                     # copy Fr*dr into the pot_array
    pot_arr      = np.cumsum(pot_arr[::-1])[::-1]  # integrate from surface to the centre
    #print 'G = ', G
    #print 'dr_arr = ', dr_arr, dr_arr.shape
    #print 'rmid_arr = ', rmid_arr, rmid_arr.shape
    #print 'drmid_arr = ', drmid_arr, drmid_arr.shape
    #print 'r3mid_arr = ', r3mid_arr, r3mid_arr.shape
    #print 'dr3mid_arr = ', dr3mid_arr, dr3mid_arr.shape
    #print 'dMrmid_arr = ', dMrmid_arr, dMrmid_arr.shape
    #print 'Mrmid_arr = ', Mrmid_arr, Mrmid_arr.shape
    #print 'Frmid_arr = ', Frmid_arr, Frmid_arr.shape
    #print 'pot_arr = ', pot_arr, pot_arr.shape
    return (pot_arr, Mrmid_arr, Frmid_arr)

def gauss_dens_prof(Rsph, Msph, rho_rat, Nr):
    rarr   = np.linspace(0.0, Rsph+Rsph/Nr, Nr+1)
    sig_R  = Rsph / np.sqrt(-np.log(rho_rat)) # characteristic radius
    rho_rarr = np.exp(-rarr*rarr/(sig_R*sig_R))
    # calculate mass of the sphere with temporary density profile
    (pot_rarr, Mrmid_rarr, Frmid_rarr) = calc_sphsym_pot(rarr, rho_rarr)
    # scale the density profile to get correct sphere mass
    rho_rarr *= Msph / Mrmid_rarr[-1]
    return (rarr, rho_rarr)

In [53]:
rarr, rho_arr = gauss_dens_prof(50, 1e5, 1./3., 1000)

In [54]:
plt.figure(figsize=(7,7))
plt.plot(rarr, rho_arr)
plt.savefig('1e5density')